In [1]:
#homework 1 decision tree v3

In [2]:
import pandas as pd #for manipulating the csv data
import numpy as np #for mathematical calculation

In [3]:
#PlayTennis.csv is the data set from the lecture and also found on Kaggle
train_data_m = pd.read_csv("TrainPlayTennis.csv") #importing the dataset from the disk

train_data_m.head() #viewing some row of the dataset

,Outlook,Temperature,Humidity,Wind,Play Tennis
0,Sunny,Hot,High,Weak,No
1,Sunny,Hot,High,Strong,No
2,Overcast,Hot,High,Weak,Yes
3,Rain,Mild,High,Weak,Yes
4,Rain,Cool,Normal,Weak,Yes


In [4]:
label_string='Play Tennis' #name of pandas label column
label_class_list = ['Yes', 'No'] #set of possible values of the label

In [5]:
#entropy function
# E(S) = Sum (-pi * lg(pi)) where i is each class
#Method description:
#train_data: a pandas dataframe/dataset
#label: string, name of the label of the dataframe (=Play Tennis)
#class_list: list, unique classes of the label (=[Yes, No]).
#returns: float, calculated entropy of the whole dataframe (=0.94)

def calc_total_entropy(train_data, label, class_list):
    total_row = train_data.shape[0] #the total size of the dataset
    total_entropy = 0 #initialize entropy
    
    for c in class_list: #for each class in the label
        total_class_count = train_data[train_data[label] == c].shape[0] #number of the class
        total_class_entr = - (total_class_count/total_row)*np.log2(total_class_count/total_row) #entropy of the class
        total_entropy += total_class_entr #adding the class entropy to the total entropy of the dataset
    
    return total_entropy

In [6]:
total_entropy = calc_total_entropy(train_data_m, label_string, label_class_list )
total_entropy

0.9402859586706311

In [7]:
# cacluate entropy for the filtered dataset
#feature
#Method description:
#feature_value_data: a pandas dataframe/dataset, which contains rows that has a specific value of a feature (Ex. Rows with only Outlook = Sunny)
#label: string, name of the label of the dataframe (=Play Tennis)
#class_list: list, unique classes of the label (=[Yes, No]).
#returns: float, calculated entropy of the feature value dataframe (Ex. for Outlook = Sunny, returns 0.971)
    
def calc_entropy(feature_value_data, label, class_list):
    class_count = feature_value_data.shape[0]
    entropy = 0
    
    for c in class_list:
        label_class_count = feature_value_data[feature_value_data[label] == c].shape[0] #row count of class c 
        entropy_class = 0
        if label_class_count != 0:
            probability_class = label_class_count/class_count #probability of the class
            entropy_class = - probability_class * np.log2(probability_class)  #entropy
        entropy += entropy_class
    return entropy

In [8]:
feature_values = train_data_m[train_data_m["Outlook"] == "Sunny"]

In [9]:
calc_entropy(feature_values, label_string, label_class_list)

0.9709505944546686

In [10]:
calc_total_entropy(feature_values, label_string, label_class_list)

0.9709505944546686

In [11]:
calc_entropy(train_data_m, label_string, label_class_list)

0.9402859586706311

In [12]:
#information gain
#IG(Y,X) = E(Y) - E(Y|X)

#feature_name: string, the name of the feature that we want to find it’s information gain (Ex. Outlook)
#train_data: a pandas dataframe/dataset
#label: string, name of the label of the dataframe (=Play Tennis)
#class_list: list, unique classes of the label (=[Yes, No]).
#returns: calculated information gain of the feature (Ex. for Outlook, returns 0.247)

def calc_info_gain(feature_name, train_data, label, class_list):
    feature_value_list = train_data[feature_name].unique() #unqiue values of the feature
    total_row = train_data.shape[0]
    feature_info = 0.0
    
    for feature_value in feature_value_list:
        feature_value_data = train_data[train_data[feature_name] == feature_value] #filtering rows with that feature_value
        feature_value_count = feature_value_data.shape[0]
        feature_value_entropy = calc_entropy(feature_value_data, label, class_list) #calculcating entropy for the feature value
        feature_value_probability = feature_value_count/total_row
        feature_info += feature_value_probability * feature_value_entropy #calculating information of the feature value
        
    return calc_total_entropy(train_data, label, class_list) - feature_info #calculating information gain by subtracting

In [13]:
calc_info_gain("Outlook", train_data_m, label_string, label_class_list)

0.24674981977443933

In [14]:
#find the most informative feature

#train_data: a pandas dataframe/dataset
#label: string, name of the label of the dataframe (=Play Tennis)
#class_list: list, unique classes of the label (=[Yes, No]).
#returns: string, the feature name


def find_most_informative_feature(train_data, label, class_list):
    feature_list = train_data.columns.drop(label) #finding the feature names in the dataset
                                            #N.B. label is not a feature, so dropping it
    max_info_gain = -1
    max_info_feature = None
    
    for feature in feature_list:  #for each feature in the dataset
        feature_info_gain = calc_info_gain(feature, train_data, label, class_list)
        if max_info_gain < feature_info_gain: #selecting feature name with highest information gain
            max_info_gain = feature_info_gain
            max_info_feature = feature
            
    return max_info_feature

In [15]:
find_most_informative_feature(train_data_m, label_string, label_class_list)

'Outlook'

In [16]:
#feature_name: string, the name of the feature that we want to add to tree and shrink dataset (Ex. Outlook)
#train_data: a pandas dataframe/dataset
#label: string, name of the label of the dataframe (=Play Tennis)
#class_list: list, unique classes of the label (=[Yes, No]).
#returns: (dictionary, dataframe), the tree node with it’s branches and the updated dataset

def generate_sub_tree(feature_name, train_data, label, class_list):
    feature_value_count_dict = train_data[feature_name].value_counts(sort=False) #dictionary of the count of unqiue feature value
    tree = {} #sub tree or node
    
    for feature_value, count in feature_value_count_dict.iteritems():
        feature_value_data = train_data[train_data[feature_name] == feature_value] #dataset with only feature_name = feature_value
        
        assigned_to_node = False #flag for tracking feature_value is pure class or not
        for c in class_list: #for each class
            class_count = feature_value_data[feature_value_data[label] == c].shape[0] #count of class c

            if class_count == count: #count of feature_value = count of class (pure class)
                tree[feature_value] = c #adding node to the tree
                train_data = train_data[train_data[feature_name] != feature_value] #removing rows with feature_value
                assigned_to_node = True
        if not assigned_to_node: #not pure class
            tree[feature_value] = "?" #should extend the node, so the branch is marked with ?
            
    return tree, train_data

In [17]:
root, train_data_sub = generate_sub_tree("Outlook", train_data_m, label_string, label_class_list)

In [18]:
root

{'Sunny': '?', 'Overcast': 'Yes', 'Rain': '?'}

In [19]:
train_data_sub

,Outlook,Temperature,Humidity,Wind,Play Tennis
0,Sunny,Hot,High,Weak,No
1,Sunny,Hot,High,Strong,No
3,Rain,Mild,High,Weak,Yes
4,Rain,Cool,Normal,Weak,Yes
5,Rain,Cool,Normal,Strong,No
7,Sunny,Mild,High,Weak,No
8,Sunny,Cool,Normal,Weak,Yes
9,Rain,Mild,Normal,Weak,Yes
10,Sunny,Mild,Normal,Strong,Yes
13,Rain,Mild,High,Strong,No


Finding the most informative feature
Making a tree node with a feature name and feature values as branches
- If pure class, adding leaf node (= Class) to the tree node
- If impure class, adding an expandable node (= ‘?’) to the tree node
Shrinking/Updating the dataset according to the pure class
Adding the node with branches into a tree
Expand the branch of the next impure class (= ‘?’) with an updated dataset
The recursion endpoint:
The dataset becomes empty after updating
There is no expandable branch (= all pure class)
Method description:
root: dictionary, the current pointed node/feature of the tree. It is contineously being updated.
prev_feature_value: Any datatype (Int or Float or String etc.) depending on the datatype of the previous feature, the previous value of the pointed node/feature
train_data: a pandas dataframe/dataset
label: string, name of the label of the dataframe (=Play Tennis)
class_list: list, unique classes of the label (=[Yes, No]).
returns: None

In [20]:
def make_tree(root, prev_feature_value, train_data, label, class_list):
    if train_data.shape[0] != 0: #if dataset becomes enpty after updating
        max_info_feature = find_most_informative_feature(train_data, label, class_list) #most informative feature
        tree, train_data = generate_sub_tree(max_info_feature, train_data, label, class_list) #getting tree node and updated dataset
        next_root = None
        
        if prev_feature_value != None: #add to intermediate node of the tree
            root[prev_feature_value] = dict()
            root[prev_feature_value][max_info_feature] = tree
            next_root = root[prev_feature_value][max_info_feature]
        else: #add to root of the tree
            root[max_info_feature] = tree
            next_root = root[max_info_feature]
        
        for node, branch in list(next_root.items()): #iterating the tree node
            if branch == "?": #if it is expandable
                feature_value_data = train_data[train_data[max_info_feature] == node] #using the updated dataset
                make_tree(next_root, node, feature_value_data, label, class_list) #recursive call with updated dataset

In [21]:
make_tree(root,None,train_data_m, label_string, label_class_list)

In [22]:
root

{'Sunny': '?',
 'Overcast': 'Yes',
 'Rain': '?',
 'Outlook': {'Sunny': {'Humidity': {'Normal': 'Yes', 'High': 'No'}},
  'Overcast': 'Yes',
  'Rain': {'Wind': {'Weak': 'Yes', 'Strong': 'No'}}}}

In [23]:
#train_data_m: a pandas dataframe/dataset
#label: string, name of the label of the dataframe (=Play Tennis)
#returns: (nested) dictionary, the decision tree
def id3(train_data_m, label):
    train_data = train_data_m.copy() #getting a copy of the dataset
    tree = {} #tree which will be updated
    class_list = train_data[label].unique() #getting unqiue classes of the label
    make_tree(tree, None, train_data_m, label, class_list) #start calling recursion
    return tree

In [24]:
tree = id3(train_data_m,"Play Tennis")

In [25]:
tree

{'Outlook': {'Sunny': {'Humidity': {'Normal': 'Yes', 'High': 'No'}},
  'Overcast': 'Yes',
  'Rain': {'Wind': {'Weak': 'Yes', 'Strong': 'No'}}}}

In [26]:
#tree: (nested) dictionary, a decision tree
#instance: series/dataframe row, a row of dataset. The row may not contain label.
#returns: Any datatype (Int or Float or String etc.) depending on the datatype of the class, the predicted classs

def predict(tree, instance):
    if not isinstance(tree, dict): #if it is leaf node
        return tree #return the value
    else:
        root_node = next(iter(tree)) #getting first key/feature name of the dictionary
        feature_value = instance[root_node] #value of the feature
        if feature_value in tree[root_node]: #checking the feature value in current tree node
            return predict(tree[root_node][feature_value], instance) #goto next feature
        else:
            return None

In [27]:
#To evaluate the model i.e. the tree we need a labeled test dataset. Then after predicting we can calculate the difference of actual and predicted value in percentage.
#tree: (nested) dictionary, a decision tree
#test_data_m: a pandas dataframe/test dataset
#returns: float, the accuray of the tree

def evaluate(tree, test_data_m, label):
    correct_preditct = 0
    wrong_preditct = 0
    for index, row in test_data_m.iterrows(): #for each row in the dataset
        result = predict(tree, test_data_m.iloc[index]) #predict the row
        if result == test_data_m[label].iloc[index]: #predicted value and expected value is same or not
            correct_preditct += 1 #increase correct count
        else:
            wrong_preditct += 1 #increase incorrect count
    accuracy = correct_preditct / (correct_preditct + wrong_preditct) #calculating accuracy
    return accuracy

In [28]:
test_data_m = pd.read_csv("TestPlayTennis.csv") #importing the dataset from the disk
test_data_m.head()

,Outlook,Temperature,Humidity,Wind,Play Tennis
0,Overcast,Mild,Normal,Weak,Yes
1,Sunny,Mild,Normal,Weak,Yes
2,Rain,Mild,Normal,Weak,Yes
3,Rain,Cool,High,Strong,No


In [29]:
evaluate(tree, test_data_m, 'Play Tennis') #evaluating the test dataset

1.0

In [35]:
test_row = test_data_m.iloc[3]
test_row

Outlook          Rain
Temperature      Cool
Humidity         High
Wind           Strong
Play Tennis        No
Name: 3, dtype: object

In [36]:
predict(tree,test_row)

'No'